# The IPAC Exoplanet Open-Source Imaging Mission Simulator User-Interface (EXOSIMS-UI) Development Guide

Written by: Rahul I. Patel

<hr>

## 1. Intro

EXOSIMS is an all-in-one sophisticated python-based simulation software that provides planetary yield estimates for space-based coronagraphic missions. EXOSIMS incorporates Monte-Carlo completeness modeling, simulates a planet-finding survey given a target list and other contraints, and optimizes the observing time during a given mission lifetime. 
 
IPAC is one of the science center for the Wide-Field Infrared Survey Telescope (WFIRST) mission, and is hosting various tools to help the community plan their observations. One of those include the coronagraphic instrument (CGI) WFIRST. To make EXOSIMS accessible to the community, we are hosting it as a backend to our front end web user interface (UI). 

## 1.1 What You Need

Here are the things you'll need to get just EXOSIMS up and running:

- EXOSIMS full code : 
> https://github.com/dsavransky/EXOSIMS. Useful to get one of the official releases https://github.com/dsavransky/EXOSIMS/releases

- Forecaster : (http://ascl.net/1701.007).  The $\texttt{fitting_parameters.h5}$ from https://github.com/chenjj2/forecaster  should be placed in $\texttt{PlanetPhysicalModel}$.
- You need an SPK ephemeris file and place it in the $\texttt{Observatory}$ subdirector of EXOSIMS. Download it form here: http://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de432s.bsp
- [Optional] NASA's Navigation and Ancillary Information Facility's SPICE system components (https://naif.jpl.nasa.gov/naif/)
- Other dependencies: https://github.com/dsavransky/EXOSIMS/blob/master/.travis.yml
- Python requirements: https://github.com/dsavransky/EXOSIMS/blob/master/requirements.txt

** The code at the moment is Python 2 compliant, and they're working on getting up to Python 3. I can't guarantee it'll work in Py3 environment**

## 1.2 Install and Quick Start

The official EXOSIMS github page has two pages to help with this. You can find them here:

- https://github.com/dsavransky/EXOSIMS/blob/master/documentation/install.rst
- https://github.com/dsavransky/EXOSIMS/blob/master/documentation/quickstart.rst

***

## 2. EXOSIMS

  There is a lot to this code. There are too many functions to cover. I dont' intend to outline all of them here, as that would be impractical. There is already documentation for each and every function and variable in the EXOSIMS Internal Control Document (ICD; https://cdn.rawgit.com/dsavransky/EXOSIMS/master/ICD/icd.pdf). 
   
   Here, my goal is to highlight the major working aspects of the code and also the interface script used for the EXOSIMS-UI

### 2.1 The Modules

The code is heavily modular, and various modules control different aspects of the simulation:

- **StarCatalog:** Details on potential targets drawn from general databases. Meta data storage. No other calculations

- **PlanetPopulation:** Encodes the probability density functions of all required planetary parameters, both physical and orbital. Does not model physical orbits or light emission/reflection -- statistics of planet occurence and properties.

- **PlanetPhysicalModel:** Models of the light emitted or reflected by planets in the wavelength bands under investigation by the current mission simulation.

- **OpticalSystem:** Contains all of the necessary information to describe the planet signal and the background noise, and calculate the integration time for a given observation. Encodes specs on instruments. 

- **ZodiacalLight:** Self-explanatory

- **BackgroundSources:** Provides density of background sources for a given target based on its coordinates and the integration depth.

- **PostProcessing:** Encodes the effects of post-processing on the data gathered in a simulated observation, and the effects on the final contrast of the simulation and is also responsible for determining whether a planet detection has occurred for a given observation.

- **Completeness:** Takes in information from the $\texttt{PlanetPopulation}$ module to determine initial completeness and update completeness values for target list stars when called upon. 

 ***
 
 The next few modules are top-level modules that oversee major aspects of the simulation and call upon the powers of the previous modules.
 
 > **TargetList:** Don't let its name fool you. This module takes in information from the $\texttt{StarCatalog}$, $\texttt{OpticalSystem}$, $\texttt{ZodiacalLight}$, $\texttt{PostProcessing}$, $\texttt{BackgroundSources}$, $\texttt{Completeness}$, $\texttt{PlanetPopulation}$, and $\texttt{PlanetPhysicalModel}$  modules to generate the target list for the simulated survey. The final target list encodes all of the same information as is provided by the $\texttt{StarCatalog}$ module.
 
 > **SimulatedUniverse:** Instantiates the $\texttt{TargetList}$ module and creates a synthetic universe by populating planetary systems about some or all of the stars in the target list.
 
 > **Observatory:** Contains all of the information specific to the space-based observatory not included in the $\texttt{OpticalSystem}$ module. 
 
 > **TimeKeeping:** The Time Keeping module is responsible for keeping track of the current mission time. It encodes only the mission start time, the mission duration, and the current time within a simulation.  
 
 > **SurveySimulation:** Performs a specific simulation based on all of the input parameters and models. This module returns the mission timeline - an ordered list of simulated observations of various targets on the target list along with their outcomes.
 
 > **SurveyEnsemble:** only task it to run multiple simulations. Parallel stuff. 

***

Note! The descriptions above are not complete. For a full description, please see the EXOSIMS ICD (https://cdn.rawgit.com/dsavransky/EXOSIMS/master/ICD/icd.pdf)




### 2.2 The Underbelly Process

The way the code works can be outlined by a flow-chart I made back in late 2016 (next Figure). The core of it is still valid, even with updates to the code. 

You'll see that the $\texttt{MissionSimulation}$ starts things up by instantiating the top level modules:

- SurveyEnsemble
- TimeKeeping
- Observatory
- SurveySimulation
- Simulated Universe

After it's been fed a list of user defined inputs from a JSON file. We'll come back to the JSON file shortly.

Then $\texttt{SimulatedUniverse}$ instantiates $\texttt{TargetList}$, which itself in turn instantiates the rest of the major modules. 

![title](FlowCharts/EXOSIMS_MissionSim.png)

 Running EXOSIMS is relatively simple. As shown in the above flow chart, the $\texttt{MissionSim}$ module is instantiated with an input parameter/script file, and then the $\texttt{SurveySimulation}$ module from $\texttt{MissionSim}$ is executed and the simulation starts running. 

The simulation then runs according to the next Flow Chart figure. Again, it's slightly out of date, but you can get the idea. 

![title](FlowCharts/EXOSIMS_runsim.png)

## 2.3 The JSON Script

The starting point of EXOSIMS lies in its input scriptfile (i.e., parameter file).
This is what is needed for the user to specify their own simulation parameters and is fed directly into the instantiation of $\texttt{MissionSim}$. 

I would break up the JSON script into five separate sections:
- General Parameters
- Science Instruments
- Star light suppression systems
- Observing Modes
- Modules

You can add as many of these as you want of the parameters that are designated as general parameters. You may want to keep determine which parameters should stay fixed, compared to the ones the user can play around with. Any parameters that are not explictly defined will be given the default values (listed in the ICD).

![title](scripts/sample_JSONScript.png)

### 2.3.1 General Parameters

I usually include these at the top of the file as they don't belong to any subgroup of parameters. Here are some to name a few, but the majority can be found the ICD

    "logfile":"logfile.txt",    logfile name if you wanna give it one.
    "missionLife": 4,           Life of the mission in years. 
    "missionPortion" : 0.1 ,    Portion of the missison devoted to coronagraphic observations.
    "missionStart" : 60676.0,   Modified Julian Date of the start of the mission.
    "extendedLife" : 0,         How many years to extend original mission.
    "pupilDiam" : 2.37,         Pupil Diameters
    "shapeFac" :0.7854,         Telescope aperture shape factor
    "obscurFac" : 0.14728,      Fraction of primary obscured
    "minComp" : 0.1,            The minimum completeness level for each target to be included in the survey.
    "settlingTime" : 1,         Amount of time, in units of days, needed for observatory to settle after a repointing.
    "dMagLim" :23,              Fundamental limiting magnitude (contrast) difference between the star and planet to which the the mission will be sensitive.
    "intCutoff" :50 ,           Maximum integration time for any obs in days
    "FAP" : 3E-7,               Planet detection false alarm probability.
    "MDP" : 1E-3,               Probability that a planet detection was missed.
    

### 2.3.2 Science Instruments

Pretty self-explanatory. You can define as many instruments you want with a multitude of specifications.

    "scienceInstruments": [
                            {
                              "name": "imaging-EMCCD",
                              "QE": 0.88,
                              "CIC": 0.0013,
                              "ENF": 1.414,
                              "pitch": 1E-5,
                              "focal": 100,
                              "idark": 5E-4,
                              "sread": 0.2, 
                              "texp": 1000
                            },
                            {
                              "name": "spectro-CCD",
                              "QE": 0.88,
                              "CIC": 0.0013,
                              "sread": 3,
                              "Rs": 70,
                              "pitch": 1E-5,
                              "focal": 100,
                              "idark": 5E-4,
                              "texp": 1000 
                            }
                         ],

### 2.3.3. Star Light Suppression Systems

Describes the occulting masks and their specs. These are encoded as a list of dictionaries containing specific attributes of all starlight suppression systems. For each system, if all attributes are missing from the dictionary, they will be assigned the default values listed, or any value directly passed as input to the class constructor. In case of multiple systems, specified wavelength values (lam, deltaLam,BW) of the first system become the new default values. All attributes can be found in the ICD.

    "starlightSuppressionSystems": [
                                        {
                                          "name": "HLC-565",
                                          "lam": 565,
                                          "BW": 0.10,
                                          "IWA": 0.13,
                                          "OWA": 1.44,
                                          "occ_trans": 0.2,
                                          "core_thruput": 0.02,
                                          "core_mean_intensity": 2e-12
                                        },
                                        {
                                          "name": "SPC-800",
                                          "lam": 800,
                                          "BW": 0.18,
                                          "IWA": 0.17,
                                          "OWA": 0.64,
                                          "occ_trans": 0.3,
                                          "core_thruput": 0.04,
                                          "core_mean_intensity": 4e-12,
                                          "core_area": 0.005
                                        }
                                    ],

### 2.3.4. Observing Modes

List of dictionaries containing specific attributes of all mission observing modes. Each observing mode is a combination of an instrument and a system, operating at a given wavelength, which by default is the wavelength defined in the starlight suppression system of the observing mode. If an observing mode is operating at a different wavelength than the system default wavelength, then this new wavelength must be added to the observing mode, and the system performance will be automatically rescaled to the new wavelength. If no observing mode is defined, the default observing mode simply combines the first instrument and the first system.

Details are in the ICD

    "observingModes": [
                        {
                          "instName": "imaging-EMCCD",
                          "systName": "HLC-565",
                          "SNR":5,
                          "lam": 565,
                          "BW": 0.10,
                          "detection":true
                        },

                        {
                          "instName": "spectro-CCD",
                          "systName": "SPC-800",
                          "lam": 400,
                          "BW": 0.10,
                          "SNR":5
                        }
                    ],

### 2.3.5 Modules

Indicates which specific modules will be used for to calculate the specifics of each "universe". If you leave all of them blank, it'll run, but it will default to non-sense. StarCatalog should have something in it besides "StarCatalog". The various modules will call the specified sub-module to do the proper calculation for a specific simulation. For instance, the $\texttt{Observatory}$ module, which handles all the physical properties of the observatory (specs, thrust, fuel, etc.) will now rely on the specifics in the $\texttt{WFIRSTObservatoryL2}$ sub-module. If we wanted a Hab-Ex like observatory, we'd create another sub-module and replace it with that.

     "modules": {
                "PlanetPopulation": "KnownRVPlanets",
                "StarCatalog": "EXOCAT1",
                "OpticalSystem": "Nemati",
                "ZodiacalLight": "Stark",
                "BackgroundSources": "GalaxiesFaintStars",
                "PlanetPhysicalModel": "Forecaster",
                "Observatory": "WFIRSTObservatoryL2",
                "TimeKeeping": " ",
                "PostProcessing": " ",
                "Completeness": "GarrettCompleteness",
                "TargetList": "KnownRVPlanetsTargetList",
                "SimulatedUniverse": "KnownRVPlanetsUniverse",
                "SurveySimulation": " ",
                "SurveyEnsemble": " "
               }

In [1]:
import numpy as np
import EXOSIMS.MissionSim as msim
#np.seterr(all='ignore'); # this is to silence all the annoying numpy warnings

In [5]:
sim = msim.MissionSim('scripts/template_rpateltest_KnownRV.json')

Logging to 'rpatel_log.txt' at level 'INFO'
Imported SurveySimulation (prototype module) from EXOSIMS.Prototypes.SurveySimulation
Imported KnownRVPlanetsUniverse (specific module) from EXOSIMS.Simulat[...]se.KnownRVPlanetsUniverse
Imported TargetList (prototype module) from EXOSIMS.Prototypes.TargetList
Imported SIMBAD300Catalog (specific module) from EXOSIMS.StarCatalog.SIMBAD300Catalog
Loaded SIMBAD300.pkl star catalog
Imported Nemati (specific module) from EXOSIMS.OpticalSystem.Nemati
Imported Stark (specific module) from EXOSIMS.ZodiacalLight.Stark
Imported PostProcessing (prototype module) from EXOSIMS.Prototypes.PostProcessing
Imported GalaxiesFaintStars (specific module) from EXOSIMS.BackgroundSources.GalaxiesFaintStars
Imported GarrettCompleteness (specific module) from EXOSIMS.Completeness.GarrettCompleteness
Imported KnownRVPlanets (specific module) from EXOSIMS.PlanetPopulation.KnownRVPlanets
Imported Forecaster (specific module) from EXOSIMS.PlanetPhysicalModel.Forecaster
G

IndexError: Target list filtered to empty.

In [4]:
DRM = sim.SurveySimulation.DRM

TL = sim.TargetList
SC = sim.StarCatalog
SU = sim.SimulatedUniverse
SSim = sim.SurveySimulation
OS = sim.OpticalSystem
ZL = sim.ZodiacalLight
BS = sim.BackgroundSources
CP = sim.Completeness
PP = sim.PlanetPopulation
PM = sim.PlanetPhysicalModel

NameError: name 'sim' is not defined

In [79]:
sim.genOutSpec()

{'FAP': 3e-07,
 'FAdMag0': 15,
 'IWA': 0.130625,
 'Irange': array([  0., 180.]),
 'MDP': 0.001,
 'Mprange': array([1.000e+00, 4.131e+03]),
 'OBduration': inf,
 'OBendTimes': array([2191.5]),
 'OBnumber': 0,
 'OBstartTimes': array([0.]),
 'OWA': inf,
 'Orange': array([  0., 360.]),
 'Revision': 'Not a valid Github or SVN revision.',
 'Rprange': array([ 1. , 22.6]),
 'WA0': 0.01,
 'WAint': 0.2920353982300885,
 'arange': array([  0.1, 100. ]),
 'atts_mapping': {'BV': 'st_bmvj',
  'Bmag': 'st_bj',
  'Hmag': 'st_h',
  'Imag': 'st_ic',
  'Jmag': 'st_j',
  'Kmag': 'st_k',
  'L': 'st_lum',
  'Name': 'pl_hostname',
  'Rmag': 'st_rc',
  'Spec': 'st_spstr',
  'Umag': 'st_uj',
  'Vmag': 'st_vj',
  'dist': 'st_dist',
  'parx': 'st_plx',
  'pmdec': 'st_pmdec',
  'pmra': 'st_pmra',
  'rv': 'st_radv'},
 'charMargin': 0.15,
 'checkKeepoutEnd': True,
 'coMass': 5800.0,
 'constrainOrbits': False,
 'currentTimeAbs': 60634.0,
 'currentTimeNorm': 0.0,
 'dMag0': 15.0,
 'dMagLim': 21.5,
 'dMagint': 19.0,
 'de

In [60]:
len([DRM[x]['star_ind'] for x in range(len(DRM)) if 1 in DRM[x]['det_status']])

0

In [61]:
sim.OpticalSystem.starlightSuppressionSystems[0]['core_contrast'](sim.OpticalSystem.starlightSuppressionSystems[0]['lam'],sim.OpticalSystem.starlightSuppressionSystems[0]['IWA'])

1.000000082740371e-10

In [62]:
TL.tint0

<Quantity [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.] d>

In [67]:
OS.WA0, OS.IWA, OS.OWA

(<Quantity 0.01 arcsec>, <Quantity 0.130625 arcsec>, <Quantity inf arcsec>)

In [64]:
np.where(SU.WA > OS.WA0)

(array([ 0,  1,  2,  3,  4,  5,  7,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38,
        39, 40, 41, 42, 43, 45, 46, 48, 49, 50, 52, 53, 56, 57, 58, 59, 60,
        61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79]),)

In [70]:
SU.dMag

array([25.41384032, 19.20772911, 21.54544521, 18.90849739, 25.31967693,
       28.12198213, 16.20869723, 16.8859883 , 17.69310639, 19.33203234,
       24.00205755, 21.53239722, 17.94447849, 19.2438319 , 20.01407096,
       15.38938101, 21.49585041, 19.30176566, 18.5686514 , 19.03263667,
       21.52511101, 18.38027462, 18.9010079 , 22.4123023 , 23.15288718,
       22.78576892, 17.72043366, 19.9516401 , 18.71840689, 18.74771674,
       21.21732718, 17.14566504, 24.83375681, 32.16767299, 21.86069452,
       21.67622361, 20.53025785, 17.04401232, 18.37826663, 23.71915343,
       19.21330607, 18.68038858, 21.10517254, 22.73278175, 17.48778409,
       18.49757562, 21.05820748, 19.12366525, 23.85978281, 20.20489589,
       19.91531949, 13.38100569, 23.2082393 , 23.6004179 , 17.85202776,
       19.16681984, 17.79740399, 19.77826974, 19.5601663 , 20.94949077,
       24.08213878, 17.97808655, 19.80072243, 18.62842038, 35.41953995,
       19.51951076, 22.30944171, 20.1354713 , 19.50257008, 25.85

In [51]:
np.where(TL.L>0,TL.L, 1e-10)

array([1.00000000e-10, 1.00000000e-10, 1.00000000e-10, 5.82103218e-01,
       1.59955803e-02, 1.00000000e-10, 1.00000000e-10, 1.00000000e-10,
       2.60015956e-02, 1.00000000e-10, 1.40928880e+00, 1.23026877e+00,
       1.00000000e-10, 1.00000000e-10, 1.00000000e-10, 6.30957344e+01,
       1.00000000e-10, 1.00000000e-10, 1.00000000e-10, 1.00000000e-10,
       1.00000000e-10, 3.09741930e-01, 3.31131121e+01, 4.59198013e+00,
       3.08318795e-01, 1.12979591e+00, 3.84591782e-01, 3.73250158e+00,
       1.00000000e-10, 2.66072506e+00, 3.09741930e-01, 1.00000000e-10,
       1.00000000e-10, 1.00000000e-10, 3.89045145e+01, 1.00000000e-10,
       1.00000000e-10, 1.00000000e-10, 8.59013522e-01, 1.09900584e-01,
       1.00000000e-10, 1.00000000e-10, 1.00000000e-10, 1.23026877e+02,
       1.00000000e-10, 1.00000000e-10])

In [69]:
CP.dMagLim

19.0